In [1]:
import cv2
import numpy as np
import pandas as pd

### Video Import and Setup

In [2]:
capture = cv2.VideoCapture("C:/Users/Jimmy/Documents/Experimental_Forest/IMG_1258-720p.mp4")
total_frames = capture.get(cv2.CAP_PROP_FRAME_COUNT)
print(total_frames)
first_frame = -1
capture.set(cv2.CAP_PROP_FRAME_COUNT, first_frame)
ret, frame = capture.read()

2217.0


In [3]:
cv2.imshow("Frame 0", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Ground Truth Import

In [4]:
df = pd.read_csv('GT_IMG_1258.csv')
df

,Frame,#ofPoints,X1,Y1,X2,Y2,X3,Y3,X4,Y4,...,X6,Y6,X7,Y7,X8,Y8,X9,Y9,X10,Y10
0,0,3,596.0,700.0,590.0,364.0,523.0,301.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,593.0,702.0,590.0,364.0,524.0,301.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,593.0,703.0,591.0,364.0,524.0,301.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,592.0,705.0,591.0,364.0,524.0,301.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,3,592.0,706.0,591.0,364.0,525.0,301.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,2213,5,221.0,720.0,354.0,622.0,736.0,495.0,903.0,414.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2214,2214,5,220.0,720.0,353.0,621.0,735.0,494.0,901.0,412.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215,2215,5,219.0,720.0,351.0,620.0,734.0,492.0,900.0,411.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2216,2216,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Line and Region of Interest Functions and Example

In [5]:
def drawLine(startX, startY, endX, endY):
    YPoints = []
    XPoints = []
    dx = endX - startX
    dy = endY - startY
    steps = abs(dx) if (abs(dx) > abs(dy)) else abs(dy)
    Xinc = dx / steps
    Yinc = dy / steps
    X = startX
    Y = startY
    for i in range(int(steps)):
        XPoints.append(round(X))
        YPoints.append(round(Y))
        X += Xinc
        Y += Yinc
    return XPoints, YPoints

In [6]:
def fillROI(GT_df, frame_index):
    
    max_width = 400
    max_y = 720 # represents floor in front
    min_y = 300 # represents "horizon"
    point_count = GT_df["#ofPoints"][frame_index]
    frame_points = GT_df.iloc[frame_index, 2:(2+point_count*2)]
    Xlist = []
    Ylist = []
    Xpoints = []
    Ypoints = []
    
    for i in range(0, point_count*2-2, 2):
        # OpenCV implementation of the line for comparison
#         cv2.line(frame, (int(frame_points[i]), int(frame_points[i+1])), 
#                          (int(frame_points[i+2]), int(frame_points[i+3])), (0,0,255), 2) 
        
        Xpoints, Ypoints = drawLine(int(frame_points[i]), int(frame_points[i+1]), 
                                     int(frame_points[i+2]), int(frame_points[i+3]))
        
        # This section determines which pixels are part of the ROI (region of interest) from the labeled points
        for j in range(len(Xpoints)):
            # This linear equation serves as a base
            width = max_width - (max_y-Ypoints[j]) * 0.8721
            width = int(width//2)
            Xlist.append(Xpoints[j])
            Ylist.append(Ypoints[j])
            for k in range(1, width, 1):
                Xlist.append(Xpoints[j]+k)
                Ylist.append(Ypoints[j])
                Xlist.append(Xpoints[j]-k)
                Ylist.append(Ypoints[j])
                             
#         Xlist += Xpoints
#         Ylist += Ypoints
    return Xlist, Ylist

In [7]:
Xlist, Ylist = fillROI(df, first_frame+1)
mask_ex = frame.copy()
for i in range(len(Xlist)):
    mask_ex[Ylist[i], Xlist[i]] = [0, 0, 255]

In [8]:
cv2.imshow("Frame 0 w/ Mask", mask_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Implementation w/ Transparent Mask

In [9]:
mask = np.zeros(frame.shape[:2], dtype=np.uint8)

# Define the coordinates of the region of interest (roi_corners)
roi_corners = np.array([(x, y) for x, y in zip(Xlist, Ylist)], dtype=np.int32)
cv2.fillPoly(mask, [roi_corners], (255, 255, 255))

# Darken the shade of the mask color (light green)
dark_green = (0, 175, 0)
darkened_mask = cv2.bitwise_and(mask, mask, mask=mask)

# Create an overlay with the same size as the image
overlay = np.zeros_like(frame)
overlay[darkened_mask != 0] = dark_green

masked_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)

output = cv2.addWeighted(frame, 1, masked_overlay, 0.25, 0.5)
cv2.imshow('Output w/ Transparent Overlay', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
def applyMask(frame, Xlist, Ylist):
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    # Define the coordinates of the region of interest (roi_corners)
    roi_corners = np.array([(x, y) for x, y in zip(Xlist, Ylist)], dtype=np.int32)
    cv2.fillPoly(mask, [roi_corners], (255, 255, 255))

    # Darken the shade of the mask color (light green)
    dark_green = (0, 175, 0)
    darkened_mask = cv2.bitwise_and(mask, mask, mask=mask)

    # Create an overlay with the same size as the image
    overlay = np.zeros_like(frame)
    overlay[darkened_mask != 0] = dark_green

    masked_overlay = cv2.bitwise_and(overlay, overlay, mask=mask)

    output = cv2.addWeighted(frame, 1, masked_overlay, 0.25, 0.5)
    return output

In [ ]:
frame_index = -1
while True:
    capture.set(cv2.CAP_PROP_FRAME_COUNT, frame_index)
    isTrue, frame = capture.read()
    if (isTrue) && ((frame_index%2)==0):
        Xlist, Ylist = fillROI(df, frame_index+1)
        masked_frame = applyMask(frame, Xlist, Ylist)
        cv2.imshow('Video', masked_frame)
        frame_index += 1
        if cv2.waitKey(2) & 0xFF==ord('d'):
            break
        